In [1]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [22]:
start_date = '20221201'
end_date = '20230131'

In [23]:
query = f"""

select 
    date_format(DATE_TRUNC('month', date_parse(yyyymmdd, '%Y%m%d')), '%Y-%m-%d') month, 
    service_obj_service_name service,
    count(distinct captain_id) total_captains,
    count(distinct customer_id) total_customers,
    count(distinct case when captain_obj_device_id is null or captain_obj_device_id = '' then captain_id end) null_device_ids_captains,
    count(distinct case when customer_obj_device_device_id is null or customer_obj_device_device_id = '' then customer_id end) null_device_ids_customers,
    count(distinct case when captain_obj_device_id = '00000000' then captain_id end) ooo_captains,
    count(distinct case when customer_obj_device_device_id = '00000000' then customer_id end) ooo_customers,
    count(distinct case when captain_obj_device_id is null or captain_obj_device_id = '00000000' or captain_obj_device_id = '' then captain_id end) both_captains,
    count(distinct case when customer_obj_device_device_id is null or customer_obj_device_device_id = '00000000' or customer_obj_device_device_id = '' then customer_id end) both_customers
    
from orders.order_logs_snapshot
where
    yyyymmdd BETWEEN '{start_date}' and '{end_date}'
    and service_obj_service_name in ('Link', 'Auto')
    and order_status = 'dropped' --and event_type = 'dropped'
group by 1,2

"""

In [24]:
df_2022_23 = pd.read_sql(query, conn)

In [25]:
df4 = df_2022_23

In [11]:
df1

,month,service,total_captains,total_customers,null_device_ids_captains,null_device_ids_customers,ooo_captains,ooo_customers,both_captains,both_customers
0,2022-03-01,Link,265145,3291441,81,11786,16,455,97,12241
1,2022-01-01,Link,217041,2216174,12,5428,9,395,21,5823
2,2022-02-01,Auto,142281,1281796,3,2218,2,190,5,2408
3,2022-02-01,Link,219896,2440227,12,7498,13,394,25,7892
4,2022-01-01,Auto,138225,1106173,4,2150,5,185,9,2335
5,2022-03-01,Auto,156017,1729812,20,4545,7,227,27,4772


In [16]:
df2

,month,service,total_captains,total_customers,null_device_ids_captains,null_device_ids_customers,ooo_captains,ooo_customers,both_captains,both_customers
0,2022-07-01,Auto,215956,2914867,26,7990,5,144,31,8134
1,2022-05-01,Auto,170784,2379161,26,5753,5,256,31,6009
2,2022-06-01,Link,373885,4306387,144,12217,27,347,171,12564
3,2022-05-01,Link,315690,3996276,106,14361,23,418,129,14779
4,2022-06-01,Auto,197435,2673808,32,6297,4,187,36,6484
5,2022-04-01,Auto,162048,1971882,23,4102,8,246,31,4348
6,2022-07-01,Link,405818,4398837,166,11780,25,294,191,12074
7,2022-04-01,Link,273469,3551945,96,11476,19,445,115,11921


In [21]:
df3

,month,service,total_captains,total_customers,null_device_ids_captains,null_device_ids_customers,ooo_captains,ooo_customers,both_captains,both_customers
0,2022-11-01,Auto,236463,2842391,31,2093,4,67,35,2160
1,2022-08-01,Link,434042,4592165,213,13956,19,268,232,14224
2,2022-10-01,Auto,235522,2997727,21,3326,6,75,27,3401
3,2022-09-01,Link,461362,4613730,179,13014,16,190,195,13204
4,2022-11-01,Link,473330,4791214,149,3266,13,137,162,3403
5,2022-09-01,Auto,238645,2975265,28,9860,8,90,36,9950
6,2022-08-01,Auto,224865,3041355,23,15339,8,136,31,15475
7,2022-10-01,Link,441352,4727901,146,5433,15,170,161,5603


In [26]:
df4

,month,service,total_captains,total_customers,null_device_ids_captains,null_device_ids_customers,ooo_captains,ooo_customers,both_captains,both_customers
0,2022-12-01,Link,482366,5191038,140,3495,12,142,152,3637
1,2023-01-01,Auto,247427,3098255,30,2125,5,49,35,2174
2,2022-12-01,Auto,243342,3157532,31,1956,4,58,35,2014
3,2023-01-01,Link,459615,5041244,126,3641,10,115,136,3756


In [27]:
frames = [df1, df2, df3, df4]
result = pd.concat(frames)

In [28]:
result.to_csv("result.csv", index = False)

In [9]:
model = f"""

select 
    distinct captain_obj_device_model model, 'Captain' User
from orders.order_logs_snapshot
where
    yyyymmdd BETWEEN '{start_date}' and '{end_date}'
    and service_obj_service_name in ('Link', 'Auto')
    and order_status = 'dropped' 
    and captain_obj_device_id = '00000000'

union 

select 
    distinct customer_obj_device_model model, 'Customer' User
from orders.order_logs_snapshot
where
    yyyymmdd BETWEEN '{start_date}' and '{end_date}'
    and service_obj_service_name in ('Link', 'Auto')
    and order_status = 'dropped' 
    and customer_obj_device_device_id = '00000000'

"""

In [10]:
df_2022_23_model = pd.read_sql(model, conn)

In [11]:
df_2022_23_model

,model,User
0,ASUS_X00TD,Customer
1,PixelV1,Customer
2,SM-G928C,Customer
3,Infinix X608,Customer
4,vivo 1718,Customer
...,...,...
151,Infinix X573,Customer
152,ASUS_X00PD,Captain
153,vivo 1920,Customer
154,HTC Desire 526GPLUS dual sim,Customer


In [12]:
df_2022_23_model.to_csv("df_2022_23_device.csv", index = False)

In [13]:
df_2022_23_model['model'].unique


<bound method Series.unique of 0                        ASUS_X00TD
1                           PixelV1
2                          SM-G928C
3                      Infinix X608
4                         vivo 1718
                   ...             
151                    Infinix X573
152                      ASUS_X00PD
153                       vivo 1920
154    HTC Desire 526GPLUS dual sim
155                   Infinix X573B
Name: model, Length: 156, dtype: object>

In [17]:
unique_model = df_2022_23_model['model'].unique


In [18]:
unique_model

<bound method Series.unique of 0                        ASUS_X00TD
1                           PixelV1
2                          SM-G928C
3                      Infinix X608
4                         vivo 1718
                   ...             
151                    Infinix X573
152                      ASUS_X00PD
153                       vivo 1920
154    HTC Desire 526GPLUS dual sim
155                   Infinix X573B
Name: model, Length: 156, dtype: object>